In [18]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from gensim.models import Word2Vec
from sklearn.preprocessing import LabelEncoder
from nltk.tokenize import word_tokenize
import numpy as np

In [19]:
pd.set_option('display.max_colwidth',None) 
df=pd.read_excel("updated_text_data.xlsx")
nepali_stopwords = set(stopwords.words('nepali'))
english_stopwords = set(stopwords.words('english'))

In [20]:
df.columns

Index(['गुनासो वर्ग', 'गुनासो'], dtype='object')

In [21]:
df['गुनासो वर्ग'].value_counts()

गुनासो वर्ग
वेबसाइट तथा अभिलेख व्यवस्थापन सम्बन्धी       2863
सोधपुछ, सुझाव, प्रशंसा सम्बन्धी              1899
कर्मचारी सम्वन्धी                            1384
स्वास्थ्यसँग सम्बन्धी                         899
अर्थ सबन्धी                                   856
खानेपानी सम्बन्धी                             770
सूचना तथा  संचार सम्बन्धी                     642
शान्ति सुरक्षा सम्बन्धी                       562
प्राकृतिक श्रोत/साधन सम्बन्धी                 233
लागु पदार्थ सम्बन्धी                          159
अर्थिक अनियमितता तथा भ्रष्टाचार सम्बन्धी       68
Name: count, dtype: int64

In [22]:
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove specified characters
    text = re.sub('[#\\/।(),०-९<<?!,—–’‘:\u200d]', '', text)
    # Strip double quotes
    text = text.strip('"')

    # Tokenize the text
    words=word_tokenize(text)
    # Remove stop words for both Nepali and English
    filtered_words = [word for word in words if word.lower() not in nepali_stopwords and word.lower() not in english_stopwords]
    # Join the filtered words to form the processed text
    processed_text = ' '.join(filtered_words)
    return processed_text


In [23]:
df_1=df[df['गुनासो वर्ग']=='वेबसाइट तथा अभिलेख व्यवस्थापन सम्बन्धी ']
df_2=df[df['गुनासो वर्ग']=='सोधपुछ, सुझाव, प्रशंसा सम्बन्धी'].sample(2863,replace=True)
df_2 = df_2.reset_index(drop=True)
df_3=df[df['गुनासो वर्ग']=='कर्मचारी सम्वन्धी '].sample(2863,replace=True)
df_3 = df_3.reset_index(drop=True)
df_4=df[df['गुनासो वर्ग']=='स्वास्थ्यसँग सम्बन्धी'].sample(2863,replace=True)
df_4 = df_4.reset_index(drop=True)
df_5=df[df['गुनासो वर्ग']=='अर्थ सबन्धी '].sample(2863,replace=True)
df_5 = df_5.reset_index(drop=True)
df_6=df[df['गुनासो वर्ग']=='खानेपानी सम्बन्धी '].sample(2863,replace=True)
df_6 = df_6.reset_index(drop=True)
df_7=df[df['गुनासो वर्ग']=='सूचना तथा  संचार सम्बन्धी'].sample(2863,replace=True)
df_7 = df_7.reset_index(drop=True)
df_8=df[df['गुनासो वर्ग']=='शान्ति सुरक्षा सम्बन्धी '].sample(2863,replace=True)
df_8 = df_8.reset_index(drop=True)
df_9=df[df['गुनासो वर्ग']=='प्राकृतिक श्रोत/साधन सम्बन्धी'].sample(2863,replace=True)
df_9 =df_9.reset_index(drop=True)
df_10=df[df['गुनासो वर्ग']=='लागु पदार्थ सम्बन्धी '].sample(2863,replace=True)
df_10 = df_10.reset_index(drop=True)
df_11=df[df['गुनासो वर्ग']=='अर्थिक अनियमितता तथा भ्रष्टाचार सम्बन्धी '].sample(2863,replace=True)
df_11 = df_11.reset_index(drop=True)



In [24]:
df_final=pd.concat([df_1,df_2,df_3,df_4,df_5,df_6,df_7,df_8,df_9,df_10,df_11],axis=0)

In [25]:
df_final['गुनासो वर्ग'].value_counts()

गुनासो वर्ग
वेबसाइट तथा अभिलेख व्यवस्थापन सम्बन्धी       2863
सोधपुछ, सुझाव, प्रशंसा सम्बन्धी              2863
कर्मचारी सम्वन्धी                            2863
स्वास्थ्यसँग सम्बन्धी                        2863
अर्थ सबन्धी                                  2863
खानेपानी सम्बन्धी                            2863
सूचना तथा  संचार सम्बन्धी                    2863
शान्ति सुरक्षा सम्बन्धी                      2863
प्राकृतिक श्रोत/साधन सम्बन्धी                2863
लागु पदार्थ सम्बन्धी                         2863
अर्थिक अनियमितता तथा भ्रष्टाचार सम्बन्धी     2863
Name: count, dtype: int64

In [26]:
df_final['गुनासो']=df_final['गुनासो'].apply(preprocess_text)

In [28]:
model_path = "../complaints/nepaliW2V_5Million.model"  # Replace with the path to your model file
model = Word2Vec.load(model_path)

In [29]:
# Function to convert text to vectors
def text_to_vector(text):
    vector = np.zeros(model.vector_size)
    count = 0
    for word in word_tokenize(text):
        if word in model.wv:
            vector += model.wv[word]
            count += 1
    if count != 0:
        vector /= count
    return vector

In [30]:
df_final['गुनासो vector'] = df_final['गुनासो'].apply(text_to_vector)
df_final.sample(1)

,गुनासो वर्ग,गुनासो,गुनासो vector
816,शान्ति सुरक्षा सम्बन्धी,सुर्खेतमा प्रहरीको कुटपिटबाट युवाको ज्यान गएको प्रहरीको कुटपिटबाट कालीकोट तिलागुफा नगरपालिकाका बर्षिया मिनी भनिने कृष्ण हमालको ज्यान गएको \r\n\r\nशुक्रबार साँझ भे क नम्बरको गाडीमा चढेर नेपालगञ्जबाट सुर्खेत आउँदै गर्दा वीरेन्द्रनगरमा सुब्बाकुना प्रहरी चेकपोष्टका असई शेरर्सिंह धामी सहितका प्रहरीले मदिरा सेवन भन्दै निर्घात कुटपिट गम्भीर घाइते बनाएको गम्भीर अवस्थामा हिजो शनिबार मृत्यु प्रहरीको कुटाईबाट चालक सहचालक घाइते भएका मदिरा खाएको विषयमा उनीहरू बिच विवाद हुँदा प्रहरीबाट कुटिएका चालक सहचालकलाई असई धामी लगायत प्रहरीले कुट्न थालेपछि कानुनी कारवाही गर्नुस् नहान्नुस् भनेर गएका हमाललाई असईले अण्डकोष पेटमा लात हानेको \r\nप्रहरीको कुटपिटबाट मारिने हमाल कालिकोटका व्यापारी यस्ता ब्यवसायिक गतिविधि जनताको सुरक्षामा प्रत्यक्ष खट्नुपर्ने स्वयं प्रहरी कर्मचारीले सामान्य कुरालाई लिएर अपनाउनुपर्ने कानुनी कारबाही समेत नपनाइ सिधैं कुटि कुटि मान्छे मार्नुसम्मको कार्य अत्यन्तै खेदजनक बिषयमा तालुक प्रहरी प्रधान कार्यालय नक्सालको ध्यानाकर्षण हुनु जरुरत,"[-0.1843249185360037, -0.2924303404724924, -0.2972263690608088, -0.029491029685596006, 0.395877386193024, -0.2226335498620756, -0.17098689458798616, 0.34537399145774544, -0.11252742039068835, -0.31001889814215244, 0.050251959862362125, 0.1593905918096425, 0.1755282203092065, 0.35888933683127106, -0.13902211690947297, -0.2651445815397892, -0.1060202281747479, -0.12259937388007529, 0.2692199659970356, -0.06258093094365905, 0.24642385317129084, 0.3315909841679968, -0.07606472560000839, 0.33988343379605795, -0.19078270747268106, -0.18262266719539183, 0.10094168112664192, 0.40354591582436117, 0.12370441061793827, -0.21278021993348376, 0.15310130446887343, 0.22665852313730284, -0.3830825383294723, -0.052131741229095495, 0.17148649105984076, 0.12256969717564062, -0.01885923907102551, 0.28484048168174925, 0.11502761839074083, 0.19496035707707052, -0.46830082827655134, 0.23352747606695629, 0.4689543927554041, -0.2794539676804561, -0.09688915507285856, -0.2803253652622516, 0.1913034151250031, 0.19475909760476498, 0.29045284559833817, 0.47240592583839314, 0.11520905760407914, 0.38736826795211526, -0.14948485258501024, -0.11615910962573253, 0.49855239752214403, -0.03065954437770415, -0.4337606046174187, 0.07650151037523756, 0.2814054260692501, 0.08021181747317314, 0.2528784754079879, -0.04798889532976318, -0.29343273083213717, 0.2301498419197742, 0.3815815290843602, 0.1658493544891826, -0.06286950678622816, -0.11050436359946617, 0.07025271072634495, 0.21573139942483976, 0.7639636098349001, 0.23003764816676267, -0.12096737966639921, 0.16367766361800024, -0.19266844765515997, -0.31259993894491345, 0.29053012177464554, 0.05228111617034301, -0.012708049725915771, -0.2952915588437463, 0.057606427907012404, 0.1579004093480762, -0.10273861768946517, 0.03726771486690268, 0.2897193014534423, 0.5658653832622804, -0.1091642755040084, 0.190349077002611, -0.02756236890618311, -0.07192670651475055, -0.6401833182200789, -0.1417020465611131, -0.27383591474208513, -0.14953715603624004, 0.2105263782199472, 0.06672329755165265, 0.15755467013223096, 0.24699790304293856, -0.24108730220323196, 0.24004438929259778, ...]"


In [31]:
le = LabelEncoder()
y = le.fit_transform(df_final['गुनासो वर्ग'])

In [32]:
# Mapping between original labels and encoded values
label_mapping = dict(zip(le.classes_, le.transform(le.classes_)))

# Display the mapping
print("Label Mapping:")
print(label_mapping)

Label Mapping:
{'अर्थ सबन्धी ': 0, 'अर्थिक अनियमितता तथा भ्रष्टाचार सम्बन्धी ': 1, 'कर्मचारी सम्वन्धी ': 2, 'खानेपानी सम्बन्धी ': 3, 'प्राकृतिक श्रोत/साधन सम्बन्धी': 4, 'लागु पदार्थ सम्बन्धी ': 5, 'वेबसाइट तथा अभिलेख व्यवस्थापन सम्बन्धी ': 6, 'शान्ति सुरक्षा सम्बन्धी ': 7, 'सूचना तथा  संचार सम्बन्धी': 8, 'सोधपुछ, सुझाव, प्रशंसा सम्बन्धी': 9, 'स्वास्थ्यसँग सम्बन्धी': 10}


In [33]:
X = np.vstack(df_final['गुनासो vector'])

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,stratify=y)


In [35]:
# Initialize and train a Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
predictions = rf_model.predict(X_test)
print(classification_report(y_test, predictions))


              precision    recall  f1-score   support

           0       0.88      0.78      0.83       572
           1       1.00      1.00      1.00       573
           2       0.88      0.85      0.87       573
           3       0.91      0.77      0.83       573
           4       0.99      0.87      0.92       573
           5       1.00      0.97      0.98       572
           6       0.33      0.74      0.46       573
           7       0.95      0.60      0.74       573
           8       0.88      0.81      0.84       573
           9       0.80      0.65      0.71       572
          10       0.91      0.78      0.84       572

    accuracy                           0.80      6299
   macro avg       0.87      0.80      0.82      6299
weighted avg       0.87      0.80      0.82      6299



In [23]:
test_text='वेबसाइट नखुलेको'
preprocessed_text=preprocess_text(test_text)
test_text_vector=text_to_vector(preprocessed_text)
predicted_test_class=rf_model.predict([test_text_vector])
# Decode the encoded data back to original labels
decoded_data = le.inverse_transform(predicted_test_class)
decoded_data

array(['वेबसाइट तथा अभिलेख व्यवस्थापन सम्बन्धी '], dtype=object)